#Evaluating essentia tempo extractor over GiantASteps EDM datasets.

In this colabolatory notebook we are going going to show how **to load GiantSteps EDM tempo dataset**. And we are going to evalute the essentia [**tempo extractor**](https://essentia.upf.edu/reference/std_RhythmExtractor2013.html) `RhythmExtractor2013(...)` over the two versions of this dataset.

## mirdata installation

In [1]:
# !pip install git+https://github.com/mir-dataset-loaders/mirdata.git
!pip install git+https://github.com/MTG/mirdata.git@Pedro/giantsteps_tempo

  Cloning https://github.com/MTG/mirdata.git (to revision Pedro/giantsteps_tempo) to /tmp/pip-req-build-kwtfvblb
  Running command git clone -q https://github.com/MTG/mirdata.git /tmp/pip-req-build-kwtfvblb
  Running command git checkout -b Pedro/giantsteps_tempo --track origin/Pedro/giantsteps_tempo
  Switched to a new branch 'Pedro/giantsteps_tempo'
  Branch 'Pedro/giantsteps_tempo' set up to track remote branch 'Pedro/giantsteps_tempo' from 'origin'.
     |████████████████████████████████| 184kB 4.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 5.6MB 8.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
  Created wheel for mirdata: filename=mirdata-0.2.0-cp36-none-any.whl size=2242283 sha256=f59b1376367ae699feda020c098722d94295ebd9dc401191af5a6b34d1c71242
  Stor

In [2]:
import mirdata.giantsteps_tempo as gs_tempo

gs_tempo.download()


0.00B [00:00, ?B/s]

Starting to download ['annotations'] to folder /root/mir_datasets/GiantSteps_tempo
> downloading annotations


2.59MB [00:00, 3.07MB/s]



            Unfortunately the audio files of the Giant Steps Tempo dataset are not available
            for download. If you have the Giant Steps audio dataset, place the contents into
            a folder called GiantSteps_tempo with the following structure:
                > GiantSteps_tempo/
                    > giantsteps-tempo-dataset-0b7d47ba8cae59d3535a02e3db69e2cf6d0af5bb/
                    > audio/
            and copy the folder to /root/mir_datasets/GiantSteps_tempo
        


Upload to colab the audio in `/root/mir_datasets/GiantSteps_tempo`.

### With the script

In [3]:
%%shell
# pwd
# mir_datasets_path='/root/mir_datasets/GiantSteps_tempo'
# cd "$mir_datasets_path/giantsteps-tempo-dataset-0b7d47ba8cae59d3535a02e3db69e2cf6d0af5bb"
# ls

# bash audio_dl.sh
# cp -r audio ..




### From gdrive

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
%%shell
cp -r "/content/gdrive/My Drive/giantsteps_tempo/audio" /root/mir_datasets/GiantSteps_tempo

And then validate the audio and annotation files.

In [6]:

gs_tempo.validate()

Success: the dataset is complete and all files are valid.
--------------------


({}, {})

#Install essentia

In [7]:
# To run this notebook on Google Colab, uncomment and run this code

!pip install essentia




     |████████████████████████████████| 11.7MB 4.4MB/s 


In [8]:
import essentia.standard
import numpy as np


In [9]:
!pip install mir_eval
import mir_eval

In [10]:
results_v1 = []
results_v2 = []
for k, v in gs_tempo.load().items():
  tempo_v1 = v.tempo.value[0]
  tempo_v2 = v.tempo_v2.value[0] if len(v.tempo_v2.value) != 0 else None
  loader = essentia.standard.MonoLoader(filename=v.audio_path)
  audio = loader()
  tempo_extractor = essentia.standard.RhythmExtractor2013()
  tempo_estimated = tempo_extractor(audio)[0]
  print(v.title, v.genre, tempo_estimated, tempo_v1, tempo_v2)
  results_v1.append(mir_eval.tempo.detection(np.array([tempo_v1, 0]), 1.0, np.array([tempo_estimated, 0])))
  if tempo_v2 is not None:
    results_v2.append(mir_eval.tempo.detection(np.array([tempo_v2, 0]), 1.0, np.array([tempo_estimated, 0])))
  


1030011 electronica 126.0022964477539 126.0 127.0
1068430 drum-and-bass 172.2659454345703 170.0 87.0
1084996 breaks 140.00856018066406 140.0 132.0
1092771 dubstep 135.80796813964844 136.0 136.0
1114156 electronica 86.14209747314453 86.0 86.0
1118326 hard-dance 172.26553344726562 171.419 170.0
1120171 drum-and-bass 116.6196517944336 174.0 87.0
1171800 dubstep 140.03179931640625 140.0 71.0
1174239 hardcore-hard-techno 172.26547241210938 172.0 170.0
1177875 hard-dance 147.59811401367188 148.0 139.0
1183908 trance 140.0277557373047 140.0 129.0
1198571 deep-house 119.98827362060547 120.0 60.0
122772 trance 139.71176147460938 140.0 139.0
1234668 drum-and-bass 87.49574279785156 175.0 87.0
1234669 drum-and-bass 116.65306091308594 175.0 153.0
1234745 trance 139.88592529296875 140.0 71.0
1234750 trance 93.09999084472656 140.0 71.0
1240669 drum-and-bass 85.4583740234375 171.0 85.0
1240672 drum-and-bass 172.265625 174.0 87.0
1317507 drum-and-bass 86.73419189453125 174.0 87.0
1327052 dj-tools 151.0

In [20]:
results_v1, results_v2 = np.array(results_v1), np.array(results_v2)

print("Essentia extractor performs across the first version of the dataset with a precission of " + str(sum(results_v1[:,1])/len(results_v1)*100) + "%")
print("Essentia extractor performs across the second version of the dataset with a precission of " + str(sum(results_v2[:,1])/len(results_v2)*100) + "%")

Essentia extractor performs across the first version of the dataset with a precission of 54.668674698795186%
Essentia extractor performs across the second version of the dataset with a precission of 46.74735249621785%
